LAN

In [1]:
# Disjoint Set(상호배타적 집합) = Union-find 자료구조

class DisjointSet:
    parent = [] # 해당 인덱스 노드의 부모
    rank = [] # 해당 인덱스 노드의 rank
    enemy = [] # 해당 노드의 적대 관계의 루트 노드
    size = [] # 해당 노드가 루트일 경우 자신 포함 가지는 노드의 개수
    
    def __init__(self, n):
        self.parent = [i for i in range(n)] # 각자 노드의 부모는 자기 자신으로 초기화
        self.rank = [1 for _ in range(n)] # 각자의 rank는 1부터 시작
        self.size = [1 for _ in range(n)] # 각자의 size는 1부터 시작
        self.enemy = [-1 for _ in range(n)] # None일 경우 enemy가 없다는 말이다.
    
    def find(self, u): # u번 노드의 root노트의 인덱스 반환
        if u == self.parent[u]:
            return u
        return self.find(self.parent[u])
    
    def union(self, u, v): # u번 노드와 v번 노드 병합
        if u == -1 or v == -1:
            return max(u,v)
        u, v = self.find(u), self.find(v) # 각자 u,v 노드의 root노드 찾기.
        if u == v: return u
        if(self.rank[u] > self.rank[v]):
            u, v = v, u
        self.parent[u] = v # 랭크가 큰 트리의 root의 subtree로 작은 랭크의 트리가 들어간다.
        self.size[v] += self.size[u] # v노드가 루트가 되면서 v의 size를 더해준다.
        self.size[u] = 0
        if self.rank[u] == self.rank[v]: self.rank[v] += 1
        return v
        # 만약 2개의 트리의 rank가 동일할 때 rank를 1 증가시킨다.
        
    def ack(self, u, v): # u, v는 같은 트리 구성하는 지 여부
        u, v = self.find(u), self.find(v) # u의 루트, v의 루트
        if self.enemy[u] == v: # u의 적이 v와 같다 -> u와 v는 다른 트리 구성
            return False
        a = self.union(u, v) # u, v를 병합
        b = self.union(self.enemy[u], self.enemy[v]) # u의 적, v의 적을 병합
        if b == -1 or a == -1: # 아직 적에 대한 정보가 없을 때
            return True
        self.enemy[a], self.enemy[b] = b, a # a의 적은 b, b의 적은 a
        return True
    
    def dis(self, u, v): # u, v가 다른 트리에 구성되어있는지 여부
        u, v = self.find(u), self.find(v)# u의 루트, v의 루트
        if u == v: return False # u와 v가 동일 -> u와 v는 동일한 트리 구성
        a = self.union(u, self.enemy[v]) # u, v의 적을 병합
        b = self.union(self.enemy[u], v) # v, u의 적을 병합
        if b == -1 or a == -1: # 아직 적에 대한 정보가 없을 때
            return True
        self.enemy[a], self.enemy[b] = b, a # a의 적은 b, b의 적은 a
        return True
    
    def get_maxsize(self): # 최대 사이즈가 될 수 있는 수 반환
        n = len(self.size)
        answer = 0
        enemy = -1 # 적
        node_size = 0 # 현재 노드의 사이즈
        enemy_size = 0 # 현재 노드의 적의 사이즈
        for i in range(n):
            enemy = self.enemy[i]
            if enemy > i: continue
            node_size = self.size[i]
            enemy_size = 0 if enemy == -1 else self.size[enemy]
            answer += max(node_size, enemy_size)
            # 현재 노드 포함된 트리의 사이즈, 적 트리의 사이즈 중 최대값
        return answer

In [2]:
# 최소 스패닝 트리
# 크루스칼 알고리즘(Kruskal Algorithm)
# : 간선(vertex)들을 가중치 오름차순으로 정렬해서 가중치가 적은 간선부터 추가해가며 최소 스패닝 트리 완성

# 간선을 추가하면서, cycle이 생기면 안 되기 때문에 확장 할 때마다 확인한다.
# 이를 위해 간선을 이루는 2개의 정점들이 이미 확장된 정점인지 확인하기 위해 상호 배타적 집합 자료 구조를 이용한다.

adj = [[0,5,1,0,0,0,0],
       [5,0,0,1,0,3,3],
       [1,0,0,4,0,0,0],
       [0,1,4,0,5,0,3],
       [0,0,0,5,0,0,0],
       [0,3,0,0,0,0,2],
       [0,3,0,3,0,2,0]] # 인접행렬
num_V = 7 # 정점의 개수
selected = [] 

def kruskal(num_V, adj, selected):
    ret = 0
    edges = [] 
    for u in range(num_V):
        for i in range(u+1, num_V):
            if adj[u][i] != 0:
                edges.append((adj[u][i],(u, i)))
                # (weight, (간선을 이루는 두 정점))
    edges.sort() # 가중치 오름차순 정렬
    disjoint_set = DisjointSet(num_V) # 상호배타적 집합
    
    for i in range(len(edges)):
        cost = edges[i][0]
        u, v = edges[i][1] 
        if disjoint_set.find(u) == disjoint_set.find(v):
            # 두 정점 u,v가 확장된 간선들의 set 중에 같은 set에 포함될 경우 무시
            continue
        disjoint_set.union(u, v) # 아니라면 2개의 간선집합을 합친다.
        selected.append((u, v)) # 그리고 해당 간선 추가
        ret += cost # 가중치 합
    return ret
    
print("Kruskal Algorithm\nminimum spanning tree: {}, cost: {}".format(selected,kruskal(num_V, adj, selected)))
# 시간 복잡도는 O(|E|lg|E|) 
# -> for문은 간선의 개수만큼 O(|E|)  걸리나, 간선을 정렬하는데 O(|E|lg|E|)가 걸림

# 프림 알고리즘(Prim Algorithm)
# : 하나의 정점으로 시작해서, 확장할 수 있는 간선 중에 가장 작은 가중치를 지닌 간선들을 추가해가며 최소 스패닝 트리 완성

adj = [[0,5,1,0,0,0,0],
       [5,0,0,1,0,3,3],
       [1,0,0,4,0,0,0],
       [0,1,4,0,5,0,3],
       [0,0,0,5,0,0,0],
       [0,3,0,0,0,0,2],
       [0,3,0,3,0,2,0]] # 인접행렬
num_V = 7 # 정점의 개수
selected = []

import sys

def prim(num_V, adj, selected):
    added = [False for _ in range(num_V)] # 현재 정점의 트리 포함 여부
    minWeight = [sys.maxsize for _ in range(num_V)] # 해당 정점에서 확장할 수 있는 간선 중 최소 가중치
    parent = [-1 for _ in range(num_V)] # 해당 정점으로 확장 될 때 거친 바로 이전 정점
    
    ret = 0
    minWeight[0] = parent[0] = 0 # 0번 정점부터 확장 시작하기 위해 0번 정점 정보 초기화
    for it in range(num_V):
        u = -1
        for v in range(num_V):
            if not added[v] and (u == -1 or minWeight[u] > minWeight[v]):
            # 아직 추가 되지 않았고, 확장할 수 있는 간선 중 최소 가중치를 지닌 정점
                u = v
        if parent[u] != u:
            selected.append((parent[u], u)) # 확장된 간선 추가
        ret += minWeight[u] # 가중치 합
        added[u] = True # 해당 정점이 확장되었음을 표시
        for i in range(num_V):
            if adj[u][i] != 0:
                weight = adj[u][i] 
                if not added[i] and minWeight[i] > weight:
                    parent[i] = u
                    minWeight[i] = weight
        # 바로 전 확장된 정점으로 부터 다시 확장 가능한 간선 및 정점 설정
    return ret
    
print("Prim Algorithm\nminimum spanning tree: {}, cost: {}".format(selected, prim(num_V, adj, selected)))

# 시간 복잡도는 O(|V|^2 + |E|) (밀집 그래프의 경우 크루스칼보다 빠르다. ∵ 인접행렬이 밀집행렬일 때 |V|^2 ≒ |E|)
# -> 추가할 새 정점 찾는데 O(|V|) 걸리며, 모든 간선은 두번씩만 검사된다.

Kruskal Algorithm
minimum spanning tree: [(0, 2), (1, 3), (5, 6), (1, 5), (2, 3), (3, 4)], cost: 16
Prim Algorithm
minimum spanning tree: [(0, 2), (2, 3), (3, 1), (1, 5), (5, 6), (3, 4)], cost: 16


[algospot의 LAN 링크](https://algospot.com/judge/problem/read/LAN)  
  
### 문제  
근거리 네트워크(LAN, Local Area Network)는 가정, 학교, 혹은 회사 등의 제한된 공간 내의 컴퓨터들을 연결하는 네트워크입니다. 알고스팟 대학교에서는 캠퍼스의 일부 건물들만이 서로 근거리 네트워크로 연결되어 있었는데, 이번에 캠퍼스 정보화 사업의 일환으로 모든 건물을 모두 연결하려고 합니다. 모든 건물이 서로 연결되어 있다는 것은 건물 사이의 케이블을 이용해 모든 건물 간에 서로 직접/간접적으로 데이터를 주고받을 수 있다는 것을 의미합니다.  
  
문제를 단순화하기 위해, 모든 건물들은 2차원 평면 위에 있는 점이라고 가정합시다. 두 건물을 직접 연결하려면 두 건물 사이의 거리 만큼의 케이블이 필요합니다. 케이블은 항상 두 건물만을 연결할 수 있으며, 두 케이블이 교차한다고 해서 두 케이블이 직접 연결된 것은 아닙니다.  
  
각 건물들의 위치와 이미 설치된 케이블들에 대한 정보가 주어질 때, 모든 건물을 연결하기 위해 추가로 설치해야 할 케이블 길이의 최소 합을 계산하는 프로그램을 작성하세요.    
  
### 출력  
한 줄에 추가로 설치해야 할 케이블의 총 길이를 출력합니다. $10^{-7}$ 이하의 상대/절대 오차가 있는 답도 정답으로 인정합니다.  

solution의  
buildings : 각 건물의 좌표를 지니는 리스트 $(len(buildings) \leq 500)$  
cables : 케이블로 연결된 건물쌍을 지니는 리스트 $(len(cables) \leq 2000)$  
  
각 좌표의 절대값은 1000이하 이다.  

In [3]:
import math

def cal_weight(v_1, v_2): # 두 정점 간의 거리(가중치) 반환
    x_1, y_1 = v_1
    x_2, y_2 = v_2
    return math.sqrt(pow(x_1 - x_2, 2) + pow(y_1 - y_2, 2))

def prim(num_v, adj): # 프림 알고리즘
    added = [False for _ in range(num_v)] # 현재 정점의 트리 포함 여부 
    parent = [-1 for _ in range(num_v)] # 해당 정점으로 확장 될 때 거친 바로 이전 정점
    MAX_VAL = cal_weight((-1000,-1000), (1000, 1000)) # 가중치 최대값
    minWeight = [MAX_VAL for _ in range(num_v)] # 해당 정점에서 확장할 수 있는 간선 중 최소 가중치
    
    ret = 0
    minWeight[0] = parent[0] = 0 # 0번 정점부터 확장 시작하기 위해 0번 정점 정보 초기화
    for it in range(num_v):
        u = -1
        for v in range(num_v):
            if not added[v] and (u == -1 or minWeight[u] > minWeight[v]):
            # 아직 추가 되지 않았고, 확장할 수 있는 간선 중 최소 가중치를 지닌 정점
                u = v
        if parent[u] != u:
            selected.append((parent[u], u)) # 확장된 간선 추가
        ret += minWeight[u] # 가중치 합
        added[u] = True # 해당 정점이 확장되었음을 표시
        for i in range(num_v):
            if adj[u][i] != -1:
                weight = adj[u][i] 
                if not added[i] and minWeight[i] > weight:
                    parent[i] = u
                    minWeight[i] = weight
        # 바로 전 확장된 정점으로 부터 다시 확장 가능한 간선 및 정점 설정
    return ret

def solution(buildings, cables):
    num_v = len(buildings)
    adj = [[-1 for _ in range(num_v)] for _ in range(num_v)]
    for v_1 in range(num_v):
        for v_2 in range(v_1+1, num_v):
            adj[v_1][v_2] = adj[v_2][v_1] = cal_weight(buildings[v_1], buildings[v_2])
    for x,y in cables:
        adj[x][y] = adj[y][x] = 0
    # 인접행렬을 초기화한다.
    # 단, 이미 연결되어있는 빌딩 사이의 가중치는 0으로 둔다.
    return prim(num_v, adj) # 프림 알고리즘을 통해 답 반환
    
        

In [4]:
buildings = [(0,0),(0,1),(1,2)]
cables = [(0,1)]

In [5]:
solution(buildings, cables)

1.4142135623730951

In [6]:
buildings = [(-7,6),(-7,8),(10,-5),(-4,3),(10,-4),(-4,6),(-5,-10),(0,4),(-10,-7),(-6,10)]
cables = [(9,7),(7,3),(9,7),(5,0),(8,6)]

In [7]:
solution(buildings, cables)

29.70422024205609